# BSDE application : Pricing of European option with two rates  

## Formulation of the Problem 

Suppose the dynamic of the underlying asset to be : 

\begin{equation}
\frac{dS_t}{S_t}=\mu dt + \sigma dB_t
\end{equation}

and let the agent be able to lend ($r$) and borrow ($R$) money using two market accounts : 

\begin{equation}
\left\{
\begin{aligned}
d\alpha_t &= R \alpha_t dt\\
d\beta_t &= r \beta_t dt\\
\end{aligned}
\right.
\end{equation}


Let $Y_t=w(t,S_t)$ be the portfolio value at time $t$. 

\begin{equation}
-dY_t=f(t,Y_t,Z_t)dt - Z_tdW_t  
\end{equation}

where 

\begin{equation}
\left\{
\begin{aligned}
	f(t,Y_t,Z_t) & =\theta Z_t - rY + (R-r)(Y-\frac{Z_t}{\sigma})^-\\
	\theta &= \frac{r-\mu}{\sigma}\\
    Z_t=\sigma S\frac{\partial Y}{\partial S}\\
\end{aligned}
\right.
\end{equation}

________

## Discretization

<img src="../Figures/discretization_bsde.png">


__________

## Simulation

In [2]:
%pylab inline --no-import-all
import numpy as np
import scipy as sp
import pylab as pl
import matplotlib.pyplot as plt
import statistics as st
import pandas as pd
from scipy.stats import norm
import time
import warnings 


Populating the interactive namespace from numpy and matplotlib


In [3]:
"Function creating N paths for the Black-Scholes formula"

def stocks_paths(S_0,r,sigma,dt,N,m,mu):  
    S = np.zeros((m + 1, N))
    dB = np.zeros((m + 1, N))
    S[0] = S_0
    for t in range(1,m + 1):
        X=np.random.standard_normal(size=N)
        S[t]=S[t-1]*np.exp((mu-sigma*sigma/2)*dt+sigma*np.sqrt(dt)*X)
        dB[t] =  np.sqrt(dt) *X
    S_dB={'Stocks' : S,'delta_B': dB}
    return S_dB    

In [16]:
def pricing_option(mu,R,K,r,sigma,N,m,S_0,T,d):
    '''
    Function to generte stock paths.
    
    Parameters
    ==========
    S_0             : float
                      positive, initial Stock Value
    mu              : float
                      drift
    K               : float 
                      Strike price
    T               : float 
                      Maturity time 
    sigma           : float 
                      volatility 
    r               : float 
                      lending interest rate 
    R               : float 
                      borrowing interest rate 
    N               : int 
                      Number of paths generated      
    m               : int 
                      number of steps
    d               : int 
                      polynomial fit degree
    
    Returns
    =======
    V_opt : float
            Price of the European option
    '''  
    
    dt=T/m                             #Time-step
    df=1/(1+r*dt)                      #Discount factor
    theta=(r - mu)/sigma
    
    S_dB=stocks_paths(S_0,r,sigma,dt,N,m,mu)       # Dictionnary with both the delta_B and Stocks generated

    S=S_dB['Stocks']
    dB=S_dB['delta_B']

    Y=np.maximum(S[-1]-K,0)                 #price of the option at time T = Initialization for a call
   
    for t in range (m-1, 0, -1):            #Iteration over time backwardly 
        X=S[t]
        
        "Regression for Z"
        
        reg1=np.polyfit(X,Y*dB[t],d)
        Z=np.polyval(reg1,X) *  (1./ dt)
        
        "regression for Y"
        reg=np.polyfit(X, Y,d)
        J = np.polyval(reg,X)
        #Y = np.polyval(reg,X)-Y*r*dt-theta*Z*dt+np.minimum(Y-(1/sigma)*Z,0)*(R-r)*dt
        Y=df*(J-theta*Z*dt+np.minimum(Y-(1./sigma)*Z, 0)*(R-r)*dt)
  
    Y_opt= df*sum(Y)/N  
    return (Y_opt)

In [17]:
"""Pricing with BS formula"""

def black_scholes(K,r,sigma, S_0,T): 
    d1=1/(sigma*np.sqrt(T))*(np.log(S_0/K)+(r+0.5*sigma*sigma)*T)
    d2=d1-sigma*np.sqrt(T)
    dscnt=np.exp(-r*T)
    Call=-norm.cdf(d2)*K*dscnt+norm.cdf(d1)*S_0
    return (Call)

___________

## Example : 



In [48]:
start = time.time()
warnings.simplefilter('ignore', np.RankWarning)
  
    
"Parameters"
T=0.5
m=12
K=100.
S_0=100.
r=0.06
sigma=0.2
N=100000
mu=0.06
R=0.06
d=5

"We price"    

price_BSDE=pricing_option(mu,R,K,r,sigma,N,m,S_0,T,d)
price_BScholes=black_scholes(K,R,sigma, S_0,T)
print ("price ", price_BSDE) 
print ("price BScholes ", price_BScholes)

end = time.time()
print(end - start,"s") 

price  7.13737390026
price BScholes  7.15589605611
2.952974796295166 s


As expected, when r=R we get the price of a simple European Option

In [11]:
start = time.time()
warnings.simplefilter('ignore', np.RankWarning)
  
    
"Parameters"
T=0.5
m=12
K=100.
S_0=100.
r=0.04
sigma=0.2
N=10000
mu=0.06
R=0.06
d=5

"We price"    

price_BSDE=pricing_option(mu,R,K,r,sigma,N,m,S_0,T,d)
price_BScholes=black_scholes(K,R,sigma, S_0,T)
print ("price ", price_BSDE) 
print ("price BScholes ", price_BScholes)

end = time.time()
print(end - start,"s") 

price  6.24794974575
price BScholes  7.15589605611
0.2572910785675049 s


Now that $r < R$, the Z has got an mpact on the computation of Y. I have been trying to find he error, as we should get a price of 7.15. 

* First, as it is working when r=R, I thought I was saving the wrong $\Delta B_t$. But when I calculate the mean and the variance of the $\Delta B_t$ generated, I get an expected mean of 0, and a variance of dt (Indeed, $\Delta B_t \equiv \sqrt(dt) \mathcal{N}(0,1)$).

* Then, I thought the regression of Z might have been wrong, but with the Random Forest Regression I get the same problem. 
